### Data scraping

**Описание**

Архитектура -- 3 слоя, если бы это была система баз данных. На уровне ERD. Точную дату все равно вытащить можно только по ссылке.

**Импорт зависимостей**

In [1]:
# !/usr/bin/safaridriver --enable

from selenium.webdriver.common.by import By
from selenium import webdriver
from getpass import getpass

import pandas as pd
import numpy as np
import datetime
import time
import re

**Сбор ссылок**

Открываем список твитов Илона Маска:

In [2]:
driver = webdriver.Safari()  # Запускаем драйвер

# Открываем нашу страницу:
driver.get('https://x.com/search?q=from%3Aelonmusk&src=typed_query')  # TODO: можно добавить и date range
time.sleep(7)  # let it load

# Для начала вводим логин:
print('input login:')
driver.find_elements(by=By.XPATH, value='.//input[@autocomplete="username"]')[0].click()
driver.find_elements(by=By.XPATH, value='.//input[@autocomplete="username"]')[0].send_keys(getpass())

# Нажимаем на третью кнопку для перехода к вводу пароля:
driver.find_elements(by=By.XPATH, value='.//button')[2].click()
time.sleep(3)

# Вводим пароль в появившемся втором поле для этого:
print('input password:')
driver.find_elements(by=By.XPATH, value='.//input')[1].click()
driver.find_elements(by=By.XPATH, value='.//input')[1].send_keys(getpass())

# Нажимаем кнопку залогиниться:
driver.find_elements(by=By.XPATH, value='.//button')[3].click()
time.sleep(7)

input login:


 ········


input password:


 ········


Получили список твитов Илона Маска. Теперь сохраним ссылку на каждый твит. Для этого ищем ссылку + скроллим дальше. 

In [3]:
%%time

tweets_links = []
prev_len = 0

for i in range(1000):
    driver.execute_script('window.scrollBy(0, 10000)')  # скроллим вниз
    time.sleep(7)

    cur_tweets_links_elements = driver.find_elements(by=By.XPATH, value='.//div/a')

    # TODO. Почему tweets_links.extend(list(map(lambda x : x.get_attribute('href'), tweets_links))) отказался работать?
    for cur_el in cur_tweets_links_elements:
        tweets_links.append(cur_el.get_attribute('href'))

    # how many good links we found:
    good_links = [l for l in tweets_links if 'status' in l and 'elonmusk' in l and 'photo' not in l and 'analytics' not in l]
    cur_len = len(list(set(good_links)))
    print(f'{cur_len} good links found on {i + 1}th step')

    # do we need a new scroll?
    if cur_len == prev_len:
        break
    prev_len = len(list(set(good_links)))

8 good links found on 1th step
16 good links found on 2th step
26 good links found on 3th step
31 good links found on 4th step
41 good links found on 5th step
52 good links found on 6th step
61 good links found on 7th step
69 good links found on 8th step
74 good links found on 9th step
78 good links found on 10th step
81 good links found on 11th step
84 good links found on 12th step
86 good links found on 13th step
90 good links found on 14th step
94 good links found on 15th step
96 good links found on 16th step
102 good links found on 17th step
105 good links found on 18th step
109 good links found on 19th step
112 good links found on 20th step
113 good links found on 21th step
113 good links found on 22th step
CPU times: user 1.72 s, sys: 292 ms, total: 2.01 s
Wall time: 2min 44s


Сохраняем результат:

In [4]:
tweets_df = pd.DataFrame(list(set(good_links)), columns=['tweet_link'])
tweets_df['time'] = datetime.datetime.now()

In [5]:
old_tweets_df = pd.read_csv('tweet_links.csv').iloc[:,1:]

In [6]:
tweets_df = pd.concat([old_tweets_df, tweets_df]).reset_index(drop=True).drop_duplicates()

In [7]:
tweets_df.reset_index(drop=True).drop_duplicates().to_csv('tweet_links.csv')

**Сбор информации по ссылкам**

Теперь отдельно обрабатываем каждую ссылку.

In [97]:
links = pd.read_csv('tweet_links.csv').iloc[:,1].unique().tolist()

Логинимся:

In [226]:
driver = webdriver.Safari()  # Запускаем драйвер

# Открываем нашу страницу:
driver.get('https://x.com/search?q=from%3Aelonmusk&src=typed_query')  # TODO: можно добавить и date range
time.sleep(7)  # let it load

# Для начала вводим логин:
print('input login:')
driver.find_elements(by=By.XPATH, value='.//input[@autocomplete="username"]')[0].click()
driver.find_elements(by=By.XPATH, value='.//input[@autocomplete="username"]')[0].send_keys(getpass())  # are

# Нажимаем на третью кнопку для перехода к вводу пароля:
driver.find_elements(by=By.XPATH, value='.//button')[2].click()
time.sleep(3)

# Вводим пароль в появившемся втором поле для этого:
print('input password:')
driver.find_elements(by=By.XPATH, value='.//input')[1].click()
driver.find_elements(by=By.XPATH, value='.//input')[1].send_keys(getpass())

# Нажимаем кнопку залогиниться:
driver.find_elements(by=By.XPATH, value='.//button')[3].click()
time.sleep(7)

input login:


 ········


input password:


 ········


Собираем инфу:

In [251]:
%%time

scraping_res = dict()

for link_num, link in enumerate(links):
    driver.get(link)
    time.sleep(7)

    # Код ломается на твитах Маска внутри ветки, поэтому ищем индекс его твита:
    branch_users = driver.find_elements(by=By.XPATH, value='.//div[@data-testid="User-Name"]')
    for branch_user_ind in range(len(driver.find_elements(by=By.XPATH, value='.//div[@data-testid="User-Name"]'))):
        _, user_login = driver.find_elements(
            by=By.XPATH, value='.//div[@data-testid="User-Name"]'
        )[branch_user_ind].text.split('@')
        if user_login == 'elonmusk':
            break

    # Все твиты в ветке имеют одну структуру, зная индекс, достаточно легко вытаскивает содержимое:
    user_name, user_login = driver.find_elements(by=By.XPATH, value='.//div[@data-testid="User-Name"]')[branch_user_ind].text.split('@')
    tweet_text = driver.find_elements(by=By.XPATH, value='.//div[@data-testid="tweetText"]')[branch_user_ind].text
    tweet_datetime = driver.find_elements(by=By.XPATH, value='.//time')[branch_user_ind].text
    statistics = driver.find_elements(by=By.XPATH, value='.//div[@role="group"]')[branch_user_ind].get_attribute('aria-label')
    
    scraping_res[link] = (
        user_name, 
        user_login, 
        tweet_text,
        tweet_datetime,
        statistics
    )

    print(f'{link_num + 1} out of {len(links)} done')

1 out of 128 done
2 out of 128 done
3 out of 128 done
4 out of 128 done
5 out of 128 done
6 out of 128 done
7 out of 128 done
8 out of 128 done
9 out of 128 done
10 out of 128 done
11 out of 128 done
12 out of 128 done
13 out of 128 done
14 out of 128 done
15 out of 128 done
16 out of 128 done
17 out of 128 done
18 out of 128 done
19 out of 128 done
20 out of 128 done
21 out of 128 done
22 out of 128 done
23 out of 128 done
24 out of 128 done
25 out of 128 done
26 out of 128 done
27 out of 128 done
28 out of 128 done
29 out of 128 done
30 out of 128 done
31 out of 128 done
32 out of 128 done
33 out of 128 done
34 out of 128 done
35 out of 128 done
36 out of 128 done
37 out of 128 done
38 out of 128 done
39 out of 128 done
40 out of 128 done
41 out of 128 done
42 out of 128 done
43 out of 128 done
44 out of 128 done
45 out of 128 done
46 out of 128 done
47 out of 128 done
48 out of 128 done
49 out of 128 done
50 out of 128 done
51 out of 128 done
52 out of 128 done
53 out of 128 done
54

Сохраняем в формате нашего датасета. Сначала в нашем формате.

In [253]:
pd.DataFrame(
    data=scraping_res.values(), 
    index=scraping_res.keys(),
    columns=[
        'user_name', 
        'user_login',
        'tweet_text',
        'tweet_datetime',
        'statistics'
    ]
).to_csv('tweet_contents.csv')

**Переводим в формат датасета**

Наконец, заключительный этап. Приводим данные к нужной нам структуре.

In [2]:
scraped_df = pd.read_csv('tweet_contents.csv').rename(columns={'Unnamed: 0': 'link'})

Инициализируем колонки:

In [7]:
scraped_df['user_name'] = scraped_df.user_name

# Возьмем с сайта на текущий момент и введем вручную эти значения:
scraped_df['user_location'] = np.nan
scraped_df['user_description'] = np.nan
scraped_df['user_created'] = '2009-06-02 20:12:29'
scraped_df['user_followers'] = 210.1 * 1e6
scraped_df['user_friends'] = 115
scraped_df['user_favourites'] = 13503
scraped_df['user_verified'] = True

scraped_df['date'] = scraped_df['tweet_datetime']
scraped_df['text'] = scraped_df.tweet_text

# Дополнительная информация
scraped_df['source'] = np.nan  # Не можем скачать source с помощью scraping

# Preprocessing later
scraped_df['hashtags'] = np.nan
scraped_df['retweets'] = scraped_df.statistics
scraped_df['favorites'] = scraped_df.statistics

# No retweets
scraped_df['is_retweet'] = False

needed_cols = [
    'user_name',
    'user_location',
    'user_description',
    'user_created',
    'user_followers',
    'user_friends',
    'user_favourites',
    'user_verified',
    'date',
    'text',
    'hashtags',
    'source',
    'retweets',
    'favorites',
    'is_retweet'
]

res_df = scraped_df.loc[:, needed_cols]

Теперь дополнительно обрабатываем каждую, чтобы привести их к нужному формату и типу данных:

In [14]:
res_df['hashtags'] = res_df.text.str.contains('#[A-Za-z]+', regex=True)

In [45]:
res_df['retweets'] = res_df['retweets'].str.findall('\d+ reposts').explode().str.rstrip(' reposts')

In [51]:
res_df['favorites'] = res_df['favorites'].str.findall('\d+ bookmarks').explode().str.rstrip(' bookmarks')

Теперь еще обработка дат:

In [66]:
pd.to_datetime('9:00:00') + pd.Timedelta(hours=12)

Timestamp('2025-01-02 21:00:00')

In [113]:
def transform_date(row_list):
    if len(row_list) == 2:
        time = row_list[0]
        date = row_list[1]
        
    elif len(row_list) == 1:
        time = '00:00:00'
        date = row_list[0]
    else:
        return np.nan

    try:
        # Обработаем время
        if 'AM' in time:
            time = time.strip(' AM') + ':00'
        elif 'PM' in time:
            time = time.strip(' PM') + ':00'
            time = str(pd.to_datetime(time) + pd.Timedelta(hours=12))[11:]
    
        # Обработаем дату
        for mon in replace_dict:
            if mon in date:
                date = date.replace(mon, replace_dict.get(mon))
    
        date = date.replace(' ', '-').replace(',', '')
    
        return pd.to_datetime(date + ' ' + time, format='%m-%d-%Y %H:%M:%S')
    except:
        return np.nan  # invalid format

In [116]:
res_df['date'] = res_df['date'].str.split(' · ').apply(lambda x : transform_date(x))

In [118]:
res_df.to_csv('tweet_dataset.csv')